In [2]:
import pandas as pd

from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

model_path = 'D:/Github/DanhgiaDuAnVaNghenghiepTuongLai/code_ai/best_model.keras'

# Tạo từ điển ánh xạ với nhóm từ khóa cho mỗi chủ đề
technically = {
    'website': ['website', 'web'],
    'mobile-app': ['mobile app', 'app di động', 'Xây dựng ứng dụng', 'Xây dựng Ứng dụng', 'Ứng dụng di động', 'Phát triển ứng dụng', 'Xây dựng ứng dụng'],
    'game': ['game'],
    'chatbot': ['chatbot', 'chat bot'],
    'artificial-intelligence': ['trí tuệ nhân tạo'],
    'machine-learning': ['machine learning', 'máy học'],
    'deep-learning': ['deep learning', 'học sâu'],
    'computer-vision': ['computer vision', 'thị giác máy tính'],
    'big-data': ['big data', 'dữ liệu lớn', 'bigdata'],
    'data-analysis': ['data analysis', 'phân tích dữ liệu'],
    'cloud-computing': ['cloud computing', 'điện toán đám mây', 'K8s', 'kubernetes'],
    'blockchain': ['blockchain', 'chuỗi khối', 'NFT'],
    'robot': ['robot', 'robotics', 'robotic'],
    'drone': ['drone', 'flycam'],
    'vr': ['vr', 'thực tế ảo', 'virtual reality'],
}

# Hàm phân loại chủ đề
def categorize_project(project_name):
    matched_themes = []
    for theme, keywords in technically.items():
        if any(keyword.lower() in project_name.lower() for keyword in keywords):
            matched_themes.append(theme)
    return ', '.join(matched_themes) if matched_themes else 'other'

# Tải dữ liệu
pd_data = pd.read_csv('D:/Github/DanhgiaDuAnVaNghenghiepTuongLai/dataset/dataset_train_ver1.csv')
df = pd_data[['nameProject_VNI']].copy()
df['category'] = df['nameProject_VNI'].apply(categorize_project)
pd_data

# Tạo các cột chỉ báo cho mỗi chủ đề
category_dummies = pd.get_dummies(df['category'].apply(lambda x: x.split(', ')).explode()).groupby(level=0).sum()
df = pd.concat([df, category_dummies], axis=1)

# Lấy văn bản dự án và các nhãn
texts = df['nameProject_VNI'].values
categories = df.columns.difference(['nameProject_VNI', 'category'])
labels = df[categories].values

# Chuẩn bị tokenizer và padding
tokenizer = Tokenizer(num_words=10000)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
max_length = max(len(seq) for seq in sequences)
data = pad_sequences(sequences, maxlen=max_length)

# Hàm tải và dự đoán chủ đề của đề tài
def predict_project_category(project_name):
    # Tải mô hình đã huấn luyện từ đường dẫn
    model = load_model(model_path)

    # Xử lý đầu vào để phù hợp với định dạng đã huấn luyện
    sequence = tokenizer.texts_to_sequences([project_name])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    
    # Dự đoán sử dụng mô hình đã tải
    prediction = model.predict(padded_sequence)
    predicted_categories = (prediction > 0.5).astype(int)
    
    # Ánh xạ dự đoán thành nhãn chủ đề
    predicted_labels = []
    for idx, category in enumerate(categories):
        if predicted_categories[0][idx] == 1:
            predicted_labels.append(category)

    return ', '.join(predicted_labels) if predicted_labels else 'Không xác định'

In [3]:
# Ví dụ về việc dự đoán chủ đề của đề tài
project_name = "Dùng website để mua đồ chơi cho trẻ"
predicted_category = predict_project_category(project_name)
print(f"Đề tài '{project_name}' thuộc các chủ đề: {predicted_category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 303ms/step
Đề tài 'Dùng website để mua đồ chơi cho trẻ' thuộc các chủ đề: website


In [4]:
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Đường dẫn tới các tệp đã lưu
tokenizer_path = 'tokenizer.joblib'
model_path = 'best_model.keras'
categories_path = 'categories.joblib'

# Tải tokenizer và categories
tokenizer = joblib.load(tokenizer_path)
categories = joblib.load(categories_path)

# Tải mô hình
model = load_model(model_path)

# Định nghĩa hàm dự đoán chủ đề dự án
def predict_project_category(project_name):
    # Xử lý đầu vào để phù hợp với định dạng đã huấn luyện
    sequence = tokenizer.texts_to_sequences([project_name])
    max_length = max(len(seq) for seq in sequence)  # Tính chiều dài tối đa từ sequence
    padded_sequence = pad_sequences(sequence, maxlen=max_length)

    # Dự đoán sử dụng mô hình đã tải
    prediction = model.predict(padded_sequence)
    predicted_categories = (prediction > 0.5).astype(int)

    # Ánh xạ dự đoán thành nhãn chủ đề
    predicted_labels = []
    for idx, category in enumerate(categories):
        if predicted_categories[0][idx] == 1:
            predicted_labels.append(category)

    return ', '.join(predicted_labels) if predicted_labels else 'Không xác định'

In [5]:
# Sử dụng hàm dự đoán
project_name = "Xây dựng ứng dụng để quản lý thiết bị thông minh"
predicted_category = predict_project_category(project_name)
print(f"Đề tài '{project_name}' thuộc các chủ đề: {predicted_category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 354ms/step
Đề tài 'Xây dựng ứng dụng để quản lý thiết bị thông minh' thuộc các chủ đề: artificial-intelligence, big-data, blockchain, chatbot, cloud-computing, computer-vision, data-analysis, deep-learning, drone, game, iot, machine-learning, mobile-app, robot, website


In [6]:
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Đường dẫn tới các tệp đã lưu
tokenizer_path = 'tokenizer.joblib'
model_path = 'best_model.keras'
categories_path = 'categories.joblib'

# Tải tokenizer và categories
tokenizer = joblib.load(tokenizer_path)
categories = joblib.load(categories_path)

# Tải mô hình
model = load_model(model_path)

# Xác định chiều dài cố định dựa trên phân tích tập dữ liệu
fixed_max_length = 100  # Ví dụ: 50 từ

# Định nghĩa hàm dự đoán chủ đề dự án
def predict_project_category(project_name):
    # Xử lý đầu vào để phù hợp với định dạng đã huấn luyện
    sequence = tokenizer.texts_to_sequences([project_name])
    padded_sequence = pad_sequences(sequence, maxlen=fixed_max_length)

    # Dự đoán sử dụng mô hình đã tải
    prediction = model.predict(padded_sequence)
    predicted_categories = (prediction > 0.1).astype(int)

    # Ánh xạ dự đoán thành nhãn chủ đề
    predicted_labels = []
    for idx, category in enumerate(categories):
        if predicted_categories[0][idx] == 1:
            predicted_labels.append(category)

    return ', '.join(predicted_labels) if predicted_labels else 'Không xác định'

# Sử dụng hàm dự đoán
project_name = "Xây dựng hệ thống chatbot tự động tư vấn tuyển sinh"
predicted_category = predict_project_category(project_name)
print("Chủ đề dự đoán: ", predicted_category)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 312ms/step
Chủ đề dự đoán:  mobile-app, other


In [12]:
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

# Đường dẫn tới các tệp đã lưu
tokenizer_path = 'tokenizer.joblib'
model_path = 'best_model.keras'
categories_path = 'categories.joblib'

# Tải tokenizer và categories
tokenizer = joblib.load(tokenizer_path)
categories = joblib.load(categories_path)

# Tải mô hình
model = load_model(model_path)

# Đọc các cấu hình từ file
config = {}
with open('config.txt', 'r') as file:
    for line in file:
        key, value = line.strip().split(':')
        config[key] = int(value)

max_length = config.get('MaxLength')
# num_words = config.get('NumWords')

# Định nghĩa hàm dự đoán chủ đề dự án
def predict_project_category(project_name):
    # Xử lý đầu vào để phù hợp với định dạng đã huấn luyện
    sequence = tokenizer.texts_to_sequences([project_name])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)

    # Dự đoán sử dụng mô hình đã tải
    prediction = model.predict(padded_sequence)
    predicted_categories = (prediction > 0.5).astype(int)

    # Ánh xạ dự đoán thành nhãn chủ đề
    predicted_labels = []
    for idx, category in enumerate(categories):
        if predicted_categories[0][idx] == 1:
            predicted_labels.append(category)

    return ', '.join(predicted_labels) if predicted_labels else 'Không xác định'

In [8]:
# Computer Science - AI
# Xây dựng hệ thống chatbot tự động tư vấn tuyển sinh
# Dự đoán doanh thu bán hàng dựa trên phân tích dữ liệu
# Phân loại cảm xúc trong văn bản tiếng Việt
# Nhận dạng giọng nói và chuyển đổi thành văn bản tiếng Việt
# Hệ thống phát hiện gian lận giao dịch ngân hàng
# Ứng dụng AI trong phân tích y học
# Xây dựng hệ thống đề xuất sản phẩm dựa trên sở thích khách hàng
# Phân loại ảnh và nhận diện đối tượng trong môi trường thực tế
# Hệ thống quản lý học tập thông minh
# Tối ưu hóa lộ trình vận chuyển hàng hóa

# Dùng website để mua đồ chơi cho trẻ
# Dùng website để quản trị nhà máy điện
# Xây dựng ứng dụng để quản lý thiết bị thông minh - X

In [16]:
# Sử dụng hàm dự đoán
# project_name = "Xây dựng ứng dụng để quản lý thiết bị thông minh"
project_name = "Dùng website để quản trị nhà máy điện"
# project_name = "Xây dựng hệ thống chatbot tự động tư vấn tuyển sinh"
predicted_category = predict_project_category(project_name)
print(f"Đề tài: {project_name}\nThuộc các chủ đề: {predicted_category}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Đề tài: Dùng website để quản trị nhà máy điện
Thuộc các chủ đề: other
